In [1]:
from pykiwoom.kiwoom import *
import time
import pandas as pd

# 로그인
kiwoom = Kiwoom()
kiwoom.CommConnect(block=True)

In [10]:
import sqlite3

# SQLite 데이터베이스 연결
conn = sqlite3.connect('20240715_90%이상 주장 단타.db')

# track_stock 테이블에서 code 열의 모든 값을 불러오기
query = "SELECT code FROM tracked_stocks"
stock_codes_df = pd.read_sql_query(query, conn)
stock_codes = stock_codes_df['code'].tolist()

# 데이터베이스 연결 종료
conn.close()

# 결과를 저장할 빈 데이터프레임 생성
all_data = pd.DataFrame()

# 각 종목 코드에 대해 데이터 요청
for code in stock_codes:
    df = kiwoom.block_request("opt10080",
                              종목코드=code,
                              기준일자="20200424",
                              수정주가구분=1,
                              output="주식일봉차트조회",
                              next=0)
    df['종목코드'] = code  # 결과 데이터프레임에 종목 코드 추가
    all_data = pd.concat([all_data, df], ignore_index=True)
    time.sleep(10)  # 키움증권 API 요청 제한을 고려한 딜레이

{'trcode': 'opt10080', 'input': [{'주식분봉차트조회요청': ['종목코드', '틱범위', '수정주가구분']}], 'output': [{'주식분차트': ['종목코드']}, {'주식분봉차트조회': ['현재가', '거래량', '체결시간', '시가', '고가', '저가', '수정주가구분', '수정비율', '대업종구분', '소업종구분', '종목정보', '수정주가이벤트', '전일종가']}]}
{'trcode': 'opt10080', 'input': [{'주식분봉차트조회요청': ['종목코드', '틱범위', '수정주가구분']}], 'output': [{'주식분차트': ['종목코드']}, {'주식분봉차트조회': ['현재가', '거래량', '체결시간', '시가', '고가', '저가', '수정주가구분', '수정비율', '대업종구분', '소업종구분', '종목정보', '수정주가이벤트', '전일종가']}]}
{'trcode': 'opt10080', 'input': [{'주식분봉차트조회요청': ['종목코드', '틱범위', '수정주가구분']}], 'output': [{'주식분차트': ['종목코드']}, {'주식분봉차트조회': ['현재가', '거래량', '체결시간', '시가', '고가', '저가', '수정주가구분', '수정비율', '대업종구분', '소업종구분', '종목정보', '수정주가이벤트', '전일종가']}]}
{'trcode': 'opt10080', 'input': [{'주식분봉차트조회요청': ['종목코드', '틱범위', '수정주가구분']}], 'output': [{'주식분차트': ['종목코드']}, {'주식분봉차트조회': ['현재가', '거래량', '체결시간', '시가', '고가', '저가', '수정주가구분', '수정비율', '대업종구분', '소업종구분', '종목정보', '수정주가이벤트', '전일종가']}]}
{'trcode': 'opt10080', 'input': [{'주식분봉차트조회요청': ['종목코드', '틱범위', '수정주가구분']}], 'output

In [11]:
all_data

,현재가,거래량,체결시간,시가,고가,저가,수정주가구분,수정비율,대업종구분,소업종구분,종목정보,수정주가이벤트,전일종가,종목코드
0,+15000,4684,20240715153030,+15000,+15000,+15000,,,,,,,,001750
1,+15000,29499,20240715153000,+15000,+15000,+15000,,,,,,,,001750
2,+15050,590,20240715151930,+15000,+15050,+15000,,,,,,,,001750
3,+15050,7691,20240715151900,+15030,+15060,+14990,,,,,,,,001750
4,+15020,616,20240715151830,+15000,+15030,+15000,,,,,,,,001750
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21595,-16375,79,20240712140930,-16365,-16375,-16365,,,,,,,,449450
21596,-16370,394,20240712140900,-16370,-16370,-16370,,,,,,,,449450
21597,-16370,181,20240712140830,-16375,-16375,-16370,,,,,,,,449450
21598,-16380,167,20240712140800,-16370,-16380,-16370,,,,,,,,449450


In [12]:
# 원하는 열만 선택하여 출력
selected_columns = ['현재가', '거래량', '체결시간', '시가', '고가', '저가', '종목코드']
filtered_df = all_data[selected_columns]

In [13]:
# 열 이름 변경 및 순서 재정렬
all_data.rename(columns={
    '현재가': 'close',
    '거래량': 'volume',
    '체결시간': 'date',
    '시가': 'open',
    '고가': 'high',
    '저가': 'low',
    '종목코드' : 'stock_code'
}, inplace=True)

In [14]:
# 원하는 순서로 열 재정렬
reordered_columns = ['stock_code', 'date', 'open', 'high', 'close', 'low', 'volume']
reordered_df = all_data[reordered_columns]

In [15]:
reordered_df

,stock_code,date,open,high,close,low,volume
0,001750,20240715153030,+15000,+15000,+15000,+15000,4684
1,001750,20240715153000,+15000,+15000,+15000,+15000,29499
2,001750,20240715151930,+15000,+15050,+15050,+15000,590
3,001750,20240715151900,+15030,+15060,+15050,+14990,7691
4,001750,20240715151830,+15000,+15030,+15020,+15000,616
...,...,...,...,...,...,...,...
21595,449450,20240712140930,-16365,-16375,-16375,-16365,79
21596,449450,20240712140900,-16370,-16370,-16370,-16370,394
21597,449450,20240712140830,-16375,-16375,-16370,-16370,181
21598,449450,20240712140800,-16370,-16380,-16380,-16370,167


In [16]:
# 체결시간 형식 변환 함수 정의
def format_datetime(dt_str):
    return pd.to_datetime(dt_str, format='%Y%m%d%H%M%S').strftime('%Y-%m-%d %H:%M:%S')

# 체결시간 열 형식 변환
reordered_df['date'] = reordered_df['date'].apply(format_datetime)

C:\Users\euphoria\AppData\Local\Temp\ipykernel_25488\2199826333.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reordered_df['date'] = reordered_df['date'].apply(format_datetime)


In [17]:
reordered_df

,stock_code,date,open,high,close,low,volume
0,001750,2024-07-15 15:30:30,+15000,+15000,+15000,+15000,4684
1,001750,2024-07-15 15:30:00,+15000,+15000,+15000,+15000,29499
2,001750,2024-07-15 15:19:30,+15000,+15050,+15050,+15000,590
3,001750,2024-07-15 15:19:00,+15030,+15060,+15050,+14990,7691
4,001750,2024-07-15 15:18:30,+15000,+15030,+15020,+15000,616
...,...,...,...,...,...,...,...
21595,449450,2024-07-12 14:09:30,-16365,-16375,-16375,-16365,79
21596,449450,2024-07-12 14:09:00,-16370,-16370,-16370,-16370,394
21597,449450,2024-07-12 14:08:30,-16375,-16375,-16370,-16370,181
21598,449450,2024-07-12 14:08:00,-16370,-16380,-16380,-16370,167


In [18]:
# reordered_df 데이터프레임을 stock_code 별로 분리
grouped_data = reordered_df.groupby('stock_code')

# 각 그룹(종목코드별 데이터)을 딕셔너리에 저장
stock_data_dict = {stock_code: data for stock_code, data in grouped_data}

# stock_data_dict에 있는 모든 종목코드의 데이터를 과거 순서로 재배열
for stock_code in stock_data_dict.keys():
    stock_data_dict[stock_code] = stock_data_dict[stock_code].sort_values(by='date')
    stock_data_dict[stock_code][['open', 'high', 'close', 'low', 'volume']] = stock_data_dict[stock_code][['open', 'high', 'close', 'low', 'volume']].apply(pd.to_numeric, errors='coerce').abs()

In [ ]:
stock_data_dict['000660']

,stock_code,date,open,high,close,low,volume
1799,000660,2024-07-12 14:12:00,231500,232000,232000,231500,1534
1798,000660,2024-07-12 14:12:30,231500,232000,231500,231500,1740
1797,000660,2024-07-12 14:13:00,231500,232000,231500,231500,1361
1796,000660,2024-07-12 14:13:30,232000,232000,232000,231500,1324
1795,000660,2024-07-12 14:14:00,231500,232500,232000,231500,41377
...,...,...,...,...,...,...,...
904,000660,2024-07-15 15:18:30,231500,231500,231000,231000,1946
903,000660,2024-07-15 15:19:00,230500,231500,231000,230500,3297
902,000660,2024-07-15 15:19:30,231000,231500,230500,230500,3452
901,000660,2024-07-15 15:30:00,230000,230000,230000,230000,222038


In [ ]:
# 종목코드 별로 필터링된 데이터를 CSV 파일로 저장
for stock_code, data in stock_data_dict.items():
    # date, open, high, close, low, volume 열만 선택하여 저장
    data[['date', 'open', 'high', 'close', 'low', 'volume']].to_csv(f"{stock_code}.csv", index=False)

In [19]:
import sqlite3

# SQLite 데이터베이스 연결 생성 (데이터베이스가 없으면 새로 생성됩니다)
conn = sqlite3.connect('20240715_90%이상 주장 단타.db')

# 종목코드 별로 필터링된 데이터를 SQLite 데이터베이스에 저장
for stock_code, data in stock_data_dict.items():
    # date, open, high, close, low, volume 열만 선택하여 테이블에 저장
    data[['date', 'open', 'high', 'close', 'low', 'volume']].to_sql(stock_code, conn, if_exists='replace', index=False)

# 데이터베이스 연결 종료
conn.close()